In [ ]:
from model_trainer import ModelTrainer
from verification_net import VerificationNet
from stacked_mnist import StackedMNIST, DataMode

from autoencoder import AutoEncoder

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

from tqdm import tqdm
import matplotlib.pyplot as plt

from pathlib import Path
from typing import Tuple

device = torch.device("mps")
torch.mps.set_per_process_memory_fraction(0.)

trainer_file = Path("trainers/ae-basic.pkl")
model_file = Path("models/ae-basic")

latent_space_size = 64
mode = DataMode.MONO

In [ ]:
class AutoEncoderTrainer(ModelTrainer):
    def __init__(
            self, 
            model, 
            loss, 
            optimizer,
            device = torch.device("mps"),
            file_name: str | Path = model_file, 
            force_learn: bool = False
        ) -> None:
        super().__init__(model, loss, optimizer, device, file_name, force_learn)

    def get_output_from_batch(self, batch):
        x, _, _ = batch
        x = x.to(self.device)
        _, output = self.model(x)
        return x, output

In [ ]:
AE = AutoEncoder(latent_space_size=latent_space_size)
loss = nn.MSELoss()
opt = optim.Adam(AE.parameters(), lr=1e-5)
device = torch.device("mps")

trainset = StackedMNIST(train=True, mode=mode)
testset = StackedMNIST(train=False, mode=mode)

Trainer = AutoEncoderTrainer(
        model=AE, 
        loss=loss, 
        optimizer=opt, 
        file_name='models/ae-basic',
        force_learn=False
    )

# Trainer = Trainer.load_trainer(trainer_file)

train_set = DataLoader(trainset, shuffle=True, batch_size=2048)
test_set = DataLoader(testset, shuffle=True, batch_size=2048)

Trainer.print_reconstructed_img(testset)

VerifNet = VerificationNet(file_name='models/verification_model_torch_ok_copy')
Trainer.print_class_coverage_and_predictability(VerifNet, trainset)
Trainer.print_class_coverage_and_predictability(VerifNet, testset)
